# Collecting and Cleaning Data

In [ ]:
https://www.rottentomatoes.com/browse/dvd-streaming-all/

In [1]:
import requests
import pandas as pd
from scrapy.http import TextResponse
import wptools
import time
from collections import Counter

In [5]:
#reading all-the urls for all the movies from rotten tomatoes browse_all section
all_urls=[]
with open("Project_all_urls.txt", "r") as f:
    for line in f:
        all_urls.append((line.strip()))
print(len(all_urls))
all_urls[:3]

9500


['https://www.rottentomatoes.com/m/love_ranch',
 'https://www.rottentomatoes.com/m/alita_battle_angel',
 'https://www.rottentomatoes.com/m/uglydolls']

In [4]:
#function for retrieving titles and review scores from rotten tomatoes with given url
def get_reviews(url):
    response = requests.get(url)
    page = TextResponse(url = response.url, body = response.text, encoding= "utf-8")
    review = page.css('span.mop-ratings-wrap__percentage::text').re("\d+")
    titles = page.css('h1[class="mop-ratings-wrap__title mop-ratings-wrap__title--top"]::text').extract_first()
    if(len(review)==0):
        return 0; titles
    elif(len(review)==1):
        a = [review[0], 0]
        return a, titles
    else:
        return page.css('span.mop-ratings-wrap__percentage::text').re("\d+"), titles   

In [ ]:
#retrieving critic and audience score for all the movies
titles = []
critic_review = []
audience_review = []
exceptions = []
for i in all_url:
    try:
        review, title = get_reviews(i)
        titles.append(title)
        critic_review.append(review[0])
        audience_review.append(review[1])
    except requests.exceptions.RequestException:
        exceptions.append(i)
    except TypeError as error:
        exceptions.append(i)
titles

In [ ]:
print(len(titles)==len(critic_review) ==len(audience_review))
print(len(titles))

In [ ]:
df = pd.DataFrame({"Movie_title":titles, "Critic_review": critic_review, "Audience_review": audience_review})
df.head()

In [7]:
error_index = []
lookuperror = []
type_error = []
name_error = []

In [8]:
# get the infobox of an individual movie
def get_info_box(movie):
    try:
        info = wptools.page(movie).get_parse().data["infobox"]
        infobox = pd.DataFrame.from_dict(info, orient='index').transpose()
    except LookupError:
        lookuperror.append(movie)
    except TypeError:
        type_error.append(movie)
    except NameError:
        name_error.append(movie)
    except:
        error_index.append(movie)
    else:
        return infobox
# making a dataframe from all infoboxes
def mix_data(titles):
    df = pd.DataFrame()
    for i in titles:
        infobox = get_info_box(i)
        df = pd.concat([df, infobox],  ignore_index=True)
    return df

In [9]:
data = mix_data(list(titles))  

en.wikipedia.org (parse) Cut Sleeve Boys
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Molière
en.wikipedia.org (imageinfo) File:Pierre Mignard - Portrait de Je...
Molière (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Pierre Mi...
  infobox: <dict(19)> name, image, imagesize, caption, birth_name,...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Special:Se...
  pageid: 51465
  parsetree: <str(49638)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Molière
  wikibase: Q687
  wikidata_url: https://www.wikidata.org/wiki/Q687
  wikitext: <str(41224)> {{short description|17th-century French p...
}
en.wikip

In [6]:
data = pd.read_csv('fffuuu.csv')

In [10]:
data.head()

,name,Movie_title,Critic_review,Audience_review,studio,music,editing,director,producer,budget,runtime,starring,gross,cinematography,distributor
0,Love Ranch,Love Ranch,13,28,NaN,[[Chris Bacon (composer)|Chris P. Bacon]],[[Paul Hirsch (film editor)|Paul Hirsch]],[[Taylor Hackford]],{{ubl|[[David Bergstein]]|[[Lou DiBella]]|Tayl...,$25 million,117 minutes,{{ubl|[[Helen Mirren]]|[[Joe Pesci]]}},"$137,885",Kieran McGuigan,{{ubl|[[Capitol Films]]|Road Rebel}}
1,Alita: Battle Angel,Alita: Battle Angel,61,93,{{Plainlist|\r\r\n* [[20th Century Fox]]|ref| ...,[[Junkie XL|Tom Holkenborg]],{{Plainlist|\r\r\n* [[Stephen E. Rivkin]]\r\r\...,[[Robert Rodriguez]],NaN,$170 million,122 minutes,{{Plainlist|\r\r\n* [[Rosa Salazar]]\r\r\n* [[...,$404.9 million,[[Bill Pope]],20th Century Fox
2,UglyDolls,UglyDolls,27,58,{{Plainlist|\r\r\n* [[STX Entertainment|STX Fa...,[[Christopher Lennertz]],{{Plainlist|\r\r\n* Julie Rogers\r\r\n* Nolan ...,[[Kelly Asbury]],{{Plainlist|\r\r\n* [[Jane Hartwell]]\r\r\n* [...,$45–53 million,87 minutes,{{Plainlist|<!-- Twelve names from poster -->|...,$27.5 million,NaN,[[STX Entertainment]]
3,The Red Sea Diving Resort,The Red Sea Diving Resort,32,81,{{Plainlist|\r\r\n* [[Bron Studios]]\r\r\n* EM...,[[Mychael Danna]],[[Tim Squyres]],[[Gideon Raff]],{{Plainlist|\r\r\n* Aaron L. Gilbert\r\r\n* Al...,NaN,130 minutes,{{Plainlist|\r\r\n* [[Chris Evans (actor)|Chri...,NaN,[[Roberto Schaefer]],[[Netflix]]
4,NaN,Rocko's Modern Life: Static Cling,100,75,NaN,NaN,NaN,Joe Murray<br>Cosmo Serguson,Lizbeth Velasco<br>[[Raymie Muzquiz]] <small>(...,NaN,45 minutes,{{Plainlist|\r\r\n* [[Carlos Alazraqui]]\r\r\n...,NaN,NaN,[[Netflix]] <br/>[[Viacom Media Networks]]


In [8]:
data.shape

(5008, 15)

In [19]:
#resolving errors
print(len(type_error))
print(len(lookuperror))
print(len(error_index))
print(len(name_error))

432
609
0
0


In [16]:
# combining errors
errors = []
errors = lookuperror + type_error
len(errors)

In [28]:
titles = list(data.Movie_title)

In [26]:
#figuring out inicies of errors to clean the remaining data
index = []
for i in errors:
    index.append(list(data.Movie_title).index(i))
len(index)

1041

In [29]:
#cleaning the data from errors
title_refined = [i for i in titles if titles.index(i) not in index]
print(len(title_refined))
title_refined[:10]

885


['Molière',
 'I Know Who Killed Me',
 'Helvetica',
 "Who's Your Caddy?",
 'Sawdust and Tinsel',
 "Stephen Tobolowsky's Birthday Party",
 'Help!',
 'Spider-Man 3',
 'Deck the Halls',
 'Into Great Silence']

In [31]:
#cleaning the data from errors
critic_review = [data.Critic_review for i in range(len(titles)) if i not in index]
audience_review =[data.Audience_review for i in range(len(titles)) if i not in index]
audience_review[:10]
data["Audience_review"] = audience_review
data["Critic_review"] = critic_review
data["Movie_title"] = title_refined

In [12]:
data

,name,Movie_title,Critic_review,Audience_review,studio,music,editing,director,producer,budget,runtime,starring,gross,cinematography,distributor
0,Love Ranch,Love Ranch,13,28,NaN,[[Chris Bacon (composer)|Chris P. Bacon]],[[Paul Hirsch (film editor)|Paul Hirsch]],[[Taylor Hackford]],{{ubl|[[David Bergstein]]|[[Lou DiBella]]|Tayl...,$25 million,117 minutes,{{ubl|[[Helen Mirren]]|[[Joe Pesci]]}},"$137,885",Kieran McGuigan,{{ubl|[[Capitol Films]]|Road Rebel}}
1,Alita: Battle Angel,Alita: Battle Angel,61,93,{{Plainlist|\r\r\n* [[20th Century Fox]]|ref| ...,[[Junkie XL|Tom Holkenborg]],{{Plainlist|\r\r\n* [[Stephen E. Rivkin]]\r\r\...,[[Robert Rodriguez]],NaN,$170 million,122 minutes,{{Plainlist|\r\r\n* [[Rosa Salazar]]\r\r\n* [[...,$404.9 million,[[Bill Pope]],20th Century Fox
2,UglyDolls,UglyDolls,27,58,{{Plainlist|\r\r\n* [[STX Entertainment|STX Fa...,[[Christopher Lennertz]],{{Plainlist|\r\r\n* Julie Rogers\r\r\n* Nolan ...,[[Kelly Asbury]],{{Plainlist|\r\r\n* [[Jane Hartwell]]\r\r\n* [...,$45–53 million,87 minutes,{{Plainlist|<!-- Twelve names from poster -->|...,$27.5 million,NaN,[[STX Entertainment]]
3,The Red Sea Diving Resort,The Red Sea Diving Resort,32,81,{{Plainlist|\r\r\n* [[Bron Studios]]\r\r\n* EM...,[[Mychael Danna]],[[Tim Squyres]],[[Gideon Raff]],{{Plainlist|\r\r\n* Aaron L. Gilbert\r\r\n* Al...,NaN,130 minutes,{{Plainlist|\r\r\n* [[Chris Evans (actor)|Chri...,NaN,[[Roberto Schaefer]],[[Netflix]]
4,NaN,Rocko's Modern Life: Static Cling,100,75,NaN,NaN,NaN,Joe Murray<br>Cosmo Serguson,Lizbeth Velasco<br>[[Raymie Muzquiz]] <small>(...,NaN,45 minutes,{{Plainlist|\r\r\n* [[Carlos Alazraqui]]\r\r\n...,NaN,NaN,[[Netflix]] <br/>[[Viacom Media Networks]]
5,Iron Sky: The Coming Race,Iron Sky: The Coming Race,33,27,[[Iron Sky|Iron Sky Universe]] {{-}} 120dB Fil...,[[Laibach (band)|Laibach]],Joona Louhivuori,[[Timo Vuorensola]],"{{unbulleted list|Tero Kaukomaa|ref| name=""Ind...",[[Euro|€]]17 million <br />( {{USD}} 10–25 mil...,92 minutes,{{unbulleted list|[[Lara Rossi]]|[[Vladimir Bu...,NaN,Mika Orasmaa,101 Films {{-}} Scoundrel Media
6,A Dog's Journey,A Dog's Journey,50,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Otherhood,Otherhood,27,0,NaN,[[Marcelo Zarvos]],{{Plainlist|\r\r\n* Sunny Hodge \r\r\n* [[Kevi...,[[Cindy Chupack]],{{Plainlist|\r\r\n* [[Jason Michael Berman]]\r...,NaN,100 minutes,{{Plainlist|\r\r\n* [[Angela Bassett]]\r\r\n* ...,NaN,[[Declan Quinn]],[[Netflix]]
8,Charlie Says,Charlie Says,58,84,NaN,Amélie de Chassey<br>Delphine Mathieu,Emmanuelle Castro,[[Nicole Garcia]],[[Alain Attal]],$7.8 million,140 minutes,[[Jean-Pierre Bacri]],$8.8 million,[[Stéphane Fontaine]],Mars Distribution
9,Girls with Balls,Girls with Balls,38,23,NaN,Sacha Chaban,NaN,Olivier Afonso,NaN,NaN,NaN,[[Guillaume Canet]]<br>[[Denis Lavant]]<br>[[D...,NaN,NaN,[[Netflix]]
